<a href="https://colab.research.google.com/github/MitaliKambli/J025_NLP/blob/master/Classwork/NLP_SQUAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
!ls drive/"My Drive"
train = squad_json_to_dataframe_train('drive/My Drive/train-v2.0.json')

In [0]:
def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [10]:
train

,index,question,context,text,answer_start,c_id
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0
...,...,...,...,...,...,...
130314,5a7e070b70df9f001a875439,Physics has broadly agreed on the definition o...,"The term ""matter"" is used throughout physics i...",NaN,NaN,19028
130315,5a7e070b70df9f001a87543a,Who coined the term partonic matter?,"The term ""matter"" is used throughout physics i...",NaN,NaN,19028
130316,5a7e070b70df9f001a87543b,What is another name for anti-matter?,"The term ""matter"" is used throughout physics i...",NaN,NaN,19028
130317,5a7e070b70df9f001a87543c,Matter usually does not need to be used in con...,"The term ""matter"" is used throughout physics i...",NaN,NaN,19028


In [11]:
import re
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from string import digits,punctuation 
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords') 
nltk.download('averaged_perceptron_tagger')

remdigits = str.maketrans('', '', digits)
stopword = set(stopwords.words('english'))

def preprocess(str):
  str = str.lower()
  str = str.translate(remdigits)
  str = re.sub(r'[^\w\s]',' ',str)
  return str

def tokens(str):
  token = word_tokenize(str)
  words = [word for word in token if word not in stopword and word not in punctuation]
  return words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [0]:
question = train['question']
question = pd.DataFrame(question)
question['B'] = question['question'].apply(preprocess)
question['B'] = question['question'].apply(tokens)
final = question['question'].apply(tokens).tolist()

In [27]:
final

[['When', 'Beyonce', 'start', 'becoming', 'popular'],
 ['What', 'areas', 'Beyonce', 'compete', 'growing'],
 ['When',
  'Beyonce',
  'leave',
  'Destiny',
  "'s",
  'Child',
  'become',
  'solo',
  'singer'],
 ['In', 'city', 'state', 'Beyonce', 'grow'],
 ['In', 'decade', 'Beyonce', 'become', 'famous'],
 ['In', 'R', 'B', 'group', 'lead', 'singer'],
 ['What', 'album', 'made', 'worldwide', 'known', 'artist'],
 ['Who', 'managed', 'Destiny', "'s", 'Child', 'group'],
 ['When', 'Beyoncé', 'rise', 'fame'],
 ['What', 'role', 'Beyoncé', 'Destiny', "'s", 'Child'],
 ['What', 'first', 'album', 'Beyoncé', 'released', 'solo', 'artist'],
 ['When', 'Beyoncé', 'release', 'Dangerously', 'Love'],
 ['How',
  'many',
  'Grammy',
  'awards',
  'Beyoncé',
  'win',
  'first',
  'solo',
  'album'],
 ['What', 'Beyoncé', "'s", 'role', 'Destiny', "'s", 'Child'],
 ['What', 'name', 'Beyoncé', "'s", 'first', 'solo', 'album'],
 ['After',
  'second',
  'solo',
  'album',
  'entertainment',
  'venture',
  'Beyonce',
  'e

In [0]:
question = question.drop('question', 1)

In [29]:
question

,B
0,"[When, Beyonce, start, becoming, popular]"
1,"[What, areas, Beyonce, compete, growing]"
2,"[When, Beyonce, leave, Destiny, 's, Child, bec..."
3,"[In, city, state, Beyonce, grow]"
4,"[In, decade, Beyonce, become, famous]"
...,...
130314,"[Physics, broadly, agreed, definition]"
130315,"[Who, coined, term, partonic, matter]"
130316,"[What, another, name, anti-matter]"
130317,"[Matter, usually, need, used, conjunction]"


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
def document(doc):
    return doc
vectorizer = TfidfVectorizer(analyzer='word',tokenizer=document,preprocessor=document,token_pattern=None)
vectors = vectorizer.fit_transform(question['B'])
feature_names = vectorizer.get_feature_names()

In [31]:
feature_names

["''",
 "''Beautiful",
 "''Carmen",
 "''Deja",
 "''The",
 "''narrow",
 "'00s",
 "'04",
 "'06",
 "'08",
 "'1941",
 "'50s",
 "'60s",
 "'70s",
 "'73",
 "'78s",
 "'80s",
 "'90s",
 "'97",
 "'98",
 "'A",
 "'Abbas",
 "'Abdu'l-Hamid",
 "'Abdullah",
 "'Abubacer",
 "'Against",
 "'Alciphron",
 "'Ali",
 "'Always",
 "'Amblin",
 "'American",
 "'An",
 "'Armageddon",
 "'Armenia",
 "'Armenians",
 "'Arrival",
 "'Articulata",
 "'Awakenings",
 "'Batman",
 "'Batteries",
 "'Best",
 "'Bible",
 "'Bonfire",
 "'Bridge",
 "'Britain",
 "'Bronx",
 "'Buena",
 "'Catch",
 "'Chaoxian",
 "'China",
 "'Christian",
 "'City",
 "'Continental",
 "'Discoveryland",
 "'Duel",
 "'Early",
 "'Ebn",
 "'Em",
 "'Empire",
 "'Escape",
 "'Falling",
 "'Fort",
 "'From",
 "'Fuga",
 "'German",
 "'Giant",
 "'God",
 "'Great",
 "'Ideals",
 "'Indiana",
 "'Intercontinental",
 "'Interstellar",
 "'Into",
 "'Jaws",
 "'Keh",
 "'King",
 "'L",
 "'LA",
 "'Latin",
 "'Lectures",
 "'Lego",
 "'Letter",
 "'Lincoln",
 "'Little",
 "'Man",
 "'Melting",
 "'Memo

In [32]:
len(feature_names)

53012

In [33]:
#!pip install gensim

In [0]:
from gensim import models

In [0]:
features = feature_names[100:110]

In [37]:
from itertools import permutations 
perm = []
perm = list(permutations(features, 2))
print(perm)

[("'Praça", "'Preventive"), ("'Praça", "'Queen"), ("'Praça", "'Ready"), ("'Praça", "'Reflections"), ("'Praça", "'Robopocalypse"), ("'Praça", "'Roller"), ("'Praça", "'S"), ("'Praça", "'Savage'/"), ("'Praça", "'Saving"), ("'Preventive", "'Praça"), ("'Preventive", "'Queen"), ("'Preventive", "'Ready"), ("'Preventive", "'Reflections"), ("'Preventive", "'Robopocalypse"), ("'Preventive", "'Roller"), ("'Preventive", "'S"), ("'Preventive", "'Savage'/"), ("'Preventive", "'Saving"), ("'Queen", "'Praça"), ("'Queen", "'Preventive"), ("'Queen", "'Ready"), ("'Queen", "'Reflections"), ("'Queen", "'Robopocalypse"), ("'Queen", "'Roller"), ("'Queen", "'S"), ("'Queen", "'Savage'/"), ("'Queen", "'Saving"), ("'Ready", "'Praça"), ("'Ready", "'Preventive"), ("'Ready", "'Queen"), ("'Ready", "'Reflections"), ("'Ready", "'Robopocalypse"), ("'Ready", "'Roller"), ("'Ready", "'S"), ("'Ready", "'Savage'/"), ("'Ready", "'Saving"), ("'Reflections", "'Praça"), ("'Reflections", "'Preventive"), ("'Reflections", "'Queen")

In [0]:
model = models.Word2Vec(perm)

In [39]:
vocabulary = model.wv.vocab
print(vocabulary)

{"'Praça": <gensim.models.keyedvectors.Vocab object at 0x7fe34d3778d0>, "'Preventive": <gensim.models.keyedvectors.Vocab object at 0x7fe34d377978>, "'Queen": <gensim.models.keyedvectors.Vocab object at 0x7fe34d377f28>, "'Ready": <gensim.models.keyedvectors.Vocab object at 0x7fe34d377c50>, "'Reflections": <gensim.models.keyedvectors.Vocab object at 0x7fe34d3777f0>, "'Robopocalypse": <gensim.models.keyedvectors.Vocab object at 0x7fe34d377c88>, "'Roller": <gensim.models.keyedvectors.Vocab object at 0x7fe34d3779b0>, "'S": <gensim.models.keyedvectors.Vocab object at 0x7fe34d377828>, "'Savage'/": <gensim.models.keyedvectors.Vocab object at 0x7fe34d377b38>, "'Saving": <gensim.models.keyedvectors.Vocab object at 0x7fe34d3777b8>}


In [41]:
for i in perm:
  print((i[0],i[1]),model.similarity(i[0],i[1]))

("'Praça", "'Preventive") 0.14726146
("'Praça", "'Queen") -0.004664383
("'Praça", "'Ready") 0.16938126
("'Praça", "'Reflections") 0.010820278
("'Praça", "'Robopocalypse") 0.12474915
("'Praça", "'Roller") -0.13921386
("'Praça", "'S") 0.1295656
("'Praça", "'Savage'/") -0.12670642
("'Praça", "'Saving") -0.00859584
("'Preventive", "'Praça") 0.14726146
("'Preventive", "'Queen") -0.08136347
("'Preventive", "'Ready") 0.00821096
("'Preventive", "'Reflections") -0.06308557
("'Preventive", "'Robopocalypse") 0.09307803
("'Preventive", "'Roller") -0.030077128
("'Preventive", "'S") -0.09028363
("'Preventive", "'Savage'/") 0.01105845
("'Preventive", "'Saving") 0.13286328
("'Queen", "'Praça") -0.004664383
("'Queen", "'Preventive") -0.08136347
("'Queen", "'Ready") 0.099985175
("'Queen", "'Reflections") 0.031110916
("'Queen", "'Robopocalypse") 0.12836175
("'Queen", "'Roller") 0.00981259
("'Queen", "'S") 0.08313198
("'Queen", "'Savage'/") 0.012396248
("'Queen", "'Saving") 0.02000194
("'Ready", "'Praça")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
